Pre-requisites: An empty wikibase. 
In this example a wikibase in wbstack.com is being used, but the script is working on the default Wikibase api so should work as well. 

These are the steps involved

##import the Wikidata integrator

In [1]:
%%capture
!pip install wikidataintegrator
from wikidataintegrator import wdi_core, wdi_login

## Get all Wikidata properties

In [2]:
query = """
SELECT DISTINCT ?property ?propertyLabel ?propertyDescription ?propType WHERE {
   ?property wikibase:directClaim ?p ;
             wikibase:propertyType ?propType ;
             schema:description ?propertyDescription ;
             rdfs:label ?propertyLabel .
   FILTER (lang(?propertyLabel) = 'en')
   FILTER (lang(?propertyDescription) = 'en')}
"""
propertiesSparql = wdi_core.WDItemEngine.execute_sparql_query(query, as_dataframe=True)
propertiesSparql
datatype_map = {'http://wikiba.se/ontology#CommonsMedia': 'commonsMedia',
                'http://wikiba.se/ontology#ExternalId': 'external-id',
                'http://wikiba.se/ontology#GeoShape': 'geo-shape',
                'http://wikiba.se/ontology#GlobeCoordinate': 'globe-coordinate',
                'http://wikiba.se/ontology#Math': 'math',
                'http://wikiba.se/ontology#Monolingualtext': 'monolingualtext',
                'http://wikiba.se/ontology#Quantity': 'quantity',
                'http://wikiba.se/ontology#String': 'string',
                'http://wikiba.se/ontology#TabularData': 'tabular-data',
                'http://wikiba.se/ontology#Time': 'time',
                'http://wikiba.se/ontology#Url': 'url',
                'http://wikiba.se/ontology#WikibaseItem': 'wikibase-item',
                'http://wikiba.se/ontology#WikibaseLexeme': 'lexeme',
                'http://wikiba.se/ontology#WikibaseForm': 'form',
                'http://wikiba.se/ontology#WikibaseSense': 'sense',
                'http://wikiba.se/ontology#MusicalNotation': 'musical-notation',
                'http://wikiba.se/ontology#WikibaseProperty': 'wikibase-property'}
propertiesSparql['datatype']= ""
for index, row in propertiesSparql.iterrows():
  row["datatype"] = datatype_map[row["propType"]] 
propertiesSparql

,property,propType,propertyLabel,propertyDescription,datatype
0,http://www.wikidata.org/entity/P6,http://wikiba.se/ontology#WikibaseItem,head of government,"head of the executive power of this town, city...",wikibase-item
1,http://www.wikidata.org/entity/P10,http://wikiba.se/ontology#CommonsMedia,video,"relevant video. For images, use the property P...",commonsMedia
2,http://www.wikidata.org/entity/P494,http://wikiba.se/ontology#ExternalId,ICD-10,identifier in the ICD catalogue codes for dise...,external-id
3,http://www.wikidata.org/entity/P493,http://wikiba.se/ontology#ExternalId,ICD-9,identifier in the ICD catalogue codes for dise...,external-id
4,http://www.wikidata.org/entity/P179,http://wikiba.se/ontology#WikibaseItem,part of the series,series which contains the subject,wikibase-item
...,...,...,...,...,...
7990,http://www.wikidata.org/entity/P7674,http://wikiba.se/ontology#ExternalId,Czech region ID,"identifier for a region, conferred by 'Český ú...",external-id
7991,http://www.wikidata.org/entity/P7671,http://wikiba.se/ontology#ExternalId,Semion author ID,"identifier of an author or reviewer, in Semion",external-id
7992,http://www.wikidata.org/entity/P7678,http://wikiba.se/ontology#ExternalId,Wikimapia ID,link to Wikimapia item,external-id
7993,http://www.wikidata.org/entity/P7672,http://wikiba.se/ontology#ExternalId,Musiikkituottajat artist ID (chart),identifier for an artist on Musiikkituottajat ...,external-id


In [3]:
propertiesSparql.columns

Index(['property', 'propType', 'propertyLabel', 'propertyDescription',
       'datatype'],
      dtype='object')

## Login to Wikibase

In [4]:
from getpass import getpass
import pprint

wikibase = "http://35.205.156.230"
api = wikibase+":8181/w/api.php"
sparql = wikibase+":8282/proxy/wdqs/bigdata/namespace/wdq/sparql"
entityUri = wikibase.replace("https:", "http:")+"entity/"
WBUSER = getpass(prompt="username:")  
WBPASS = getpass(prompt='Enter your password: ')  
login = wdi_login.WDLogin(WBUSER, WBPASS, mediawiki_api_url=api)

username:··········
Enter your password: ··········


#### Function to create properties

In [5]:
## TODO: The mapping property in the target wikibase was made manually, but should
## be included in this script

## TODO 2: This script only considers English, but all languages supported should be added
def createProperty(login, wdprop, label, description, property_datatype):
  if wdprop != None:
    s = [wdi_core.WDUrl(wdprop, prop_nr="P1")]
  else:
    s = []
  localEntityEngine = wdi_core.WDItemEngine.wikibase_item_engine_factory(api,sparql)
  item = localEntityEngine(data=s)
  item.set_label(label)
  item.set_description(description)
  print(item.write(login, entity_type="property", property_datatype=property_datatype))

createProperty(login, None, "wikidata property", "wikidata property mapping", "url")

#### Create Wikidata properties

In [6]:
import traceback

datatype_map = {'http://wikiba.se/ontology#CommonsMedia': 'commonsMedia',
                'http://wikiba.se/ontology#ExternalId': 'external-id',
                'http://wikiba.se/ontology#GeoShape': 'geo-shape',
                'http://wikiba.se/ontology#GlobeCoordinate': 'globe-coordinate',
                'http://wikiba.se/ontology#Math': 'math',
                'http://wikiba.se/ontology#Monolingualtext': 'monolingualtext',
                'http://wikiba.se/ontology#Quantity': 'quantity',
                'http://wikiba.se/ontology#String': 'string',
                'http://wikiba.se/ontology#TabularData': 'tabular-data',
                'http://wikiba.se/ontology#Time': 'time',
                'http://wikiba.se/ontology#Url': 'url',
                'http://wikiba.se/ontology#WikibaseItem': 'wikibase-item',
                'http://wikiba.se/ontology#WikibaseProperty': 'wikibase-property'}

for row in propertiesSparql.itertuples():
  print(row)
  try:
    if row.propType in datatype_map.keys():
       createProperty(login, row.property,row. propertyLabel, row.propertyDescription, datatype_map[row.propType])  
  except:
    print("error")
    traceback.print_exc()
    continue

    

Pandas(Index=0, property='http://www.wikidata.org/entity/P6', propType='http://wikiba.se/ontology#WikibaseItem', propertyLabel='head of government', propertyDescription='head of the executive power of this town, city, municipality, state, country, or other governmental body', datatype='wikibase-item')


/usr/local/lib/python3.6/dist-packages/wikidataintegrator/wdi_core.py:326: UserWarning: Warning: No distinct value properties found
Please set P2302 and Q21502410 in your wikibase or set `core_props` manually.
Continuing with no core_props
  "Continuing with no core_props")


P2
Pandas(Index=1, property='http://www.wikidata.org/entity/P10', propType='http://wikiba.se/ontology#CommonsMedia', propertyLabel='video', propertyDescription='relevant video. For images, use the property P18. For film trailers, qualify with "object has role" (P3831)="trailer" (Q622550)', datatype='commonsMedia')
P3
Pandas(Index=2, property='http://www.wikidata.org/entity/P494', propType='http://wikiba.se/ontology#ExternalId', propertyLabel='ICD-10', propertyDescription='identifier in the\xa0ICD catalogue codes for diseases - Version 10', datatype='external-id')
P4
Pandas(Index=3, property='http://www.wikidata.org/entity/P493', propType='http://wikiba.se/ontology#ExternalId', propertyLabel='ICD-9', propertyDescription='identifier in the ICD catalogue codes for diseases – Version 9', datatype='external-id')
P5
Pandas(Index=4, property='http://www.wikidata.org/entity/P179', propType='http://wikiba.se/ontology#WikibaseItem', propertyLabel='part of the series', propertyDescription='series

Traceback (most recent call last):
  File "<ipython-input-6-ca2fe3a2eb00>", line 21, in <module>
    createProperty(login, row.property,row. propertyLabel, row.propertyDescription, datatype_map[row.propType])
  File "<ipython-input-5-e1223da8c8cb>", line 11, in createProperty
    print(item.write(login, entity_type="property", property_datatype=property_datatype))
  File "/usr/local/lib/python3.6/dist-packages/wikidataintegrator/wdi_core.py", line 1078, in write
    raise WDApiError(json_data)
wikidataintegrator.wdi_core.WDApiError: {'error': {'code': 'modification-failed', 'info': 'Description must be no more than 250 characters long', 'messages': [{'name': 'wikibase-validator-description-too-long', 'parameters': ['250', 'practitioners of this industr...'], 'html': {'*': 'Description must be no more than 250 characters long'}}], '*': 'See http://35.205.156.230:8181/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/mailman/

P1195
Pandas(Index=1210, property='http://www.wikidata.org/entity/P4969', propType='http://wikiba.se/ontology#WikibaseItem', propertyLabel='derivative work', propertyDescription='new work of art (film, book, software, etc.) derived from major part of this work', datatype='wikibase-item')
P1196
Pandas(Index=1211, property='http://www.wikidata.org/entity/P800', propType='http://wikiba.se/ontology#WikibaseItem', propertyLabel='notable work', propertyDescription="notable scientific, artistic or literary work, or other work of significance among subject's works", datatype='wikibase-item')
P1197
Pandas(Index=1212, property='http://www.wikidata.org/entity/P802', propType='http://wikiba.se/ontology#WikibaseItem', propertyLabel='student', propertyDescription='notable student(s) of the subject individual', datatype='wikibase-item')
P1198
Pandas(Index=1213, property='http://www.wikidata.org/entity/P4101', propType='http://wikiba.se/ontology#WikibaseItem', propertyLabel='dissertation submitted to'

...

P7269
Pandas(Index=7300, property='http://www.wikidata.org/entity/P5753', propType='http://wikiba.se/ontology#WikibaseItem', propertyLabel='ideographic description sequences', propertyDescription='method to describe composition of Han characters using ideographic description characters and character components', datatype='wikibase-item')
P7270
Pandas(Index=7301, property='http://www.wikidata.org/entity/P5758', propType='http://wikiba.se/ontology#ExternalId', propertyLabel='PZS hut ID', propertyDescription='identifier for a hut on the Alpine Association of Slovenia website', datatype='external-id')
P7271
Pandas(Index=7302, property='http://www.wikidata.org/entity/P5752', propType='http://wikiba.se/ontology#ExternalId', propertyLabel='Syndikat writer ID', propertyDescription='identifier for a writer on the Syndikat website', datatype='external-id')
P7272
Pandas(Index=7303, property='http://www.wikidata.org/entity/P5756', propType='http://wikiba.se/ontology#ExternalId', propertyLabel='ABM

Traceback (most recent call last):
  File "<ipython-input-6-ca2fe3a2eb00>", line 21, in <module>
    createProperty(login, row.property,row. propertyLabel, row.propertyDescription, datatype_map[row.propType])
  File "<ipython-input-5-e1223da8c8cb>", line 11, in createProperty
    print(item.write(login, entity_type="property", property_datatype=property_datatype))
  File "/usr/local/lib/python3.6/dist-packages/wikidataintegrator/wdi_core.py", line 1078, in write
    raise WDApiError(json_data)
wikidataintegrator.wdi_core.WDApiError: {'error': {'code': 'param-illegal', 'info': 'A datatype was expected, but either missing or not recognized.', 'messages': [{'name': 'wikibase-api-not-recognized-datatype', 'parameters': [], 'html': {'*': 'A datatype was expected, but either missing or not recognized.'}}], '*': 'See http://35.205.156.230:8181/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/mailman/listinfo/mediawiki-api-announc

P7506
Pandas(Index=7540, property='http://www.wikidata.org/entity/P8157', propType='http://wikiba.se/ontology#ExternalId', propertyLabel='MémorialGenWeb monument ID', propertyDescription='ID of a monument (war memorial) on MémorialGenWeb database', datatype='external-id')
P7507
Pandas(Index=7541, property='http://www.wikidata.org/entity/P8158', propType='http://wikiba.se/ontology#ExternalId', propertyLabel='History Colorado ID', propertyDescription='identifier for a historic structure or a building on the History Colorado website', datatype='external-id')
P7508
Pandas(Index=7542, property='http://www.wikidata.org/entity/P8148', propType='http://wikiba.se/ontology#ExternalId', propertyLabel='re:publica speaker ID', propertyDescription='identifier of a person in the re:publica conference database of speakers', datatype='external-id')
P7509
Pandas(Index=7543, property='http://www.wikidata.org/entity/P8151', propType='http://wikiba.se/ontology#ExternalId', propertyLabel='Sicilian Regional 

Traceback (most recent call last):
  File "<ipython-input-6-ca2fe3a2eb00>", line 21, in <module>
    createProperty(login, row.property,row. propertyLabel, row.propertyDescription, datatype_map[row.propType])
  File "<ipython-input-5-e1223da8c8cb>", line 11, in createProperty
    print(item.write(login, entity_type="property", property_datatype=property_datatype))
  File "/usr/local/lib/python3.6/dist-packages/wikidataintegrator/wdi_core.py", line 1078, in write
    raise WDApiError(json_data)
wikidataintegrator.wdi_core.WDApiError: {'error': {'code': 'param-illegal', 'info': 'A datatype was expected, but either missing or not recognized.', 'messages': [{'name': 'wikibase-api-not-recognized-datatype', 'parameters': [], 'html': {'*': 'A datatype was expected, but either missing or not recognized.'}}], '*': 'See http://35.205.156.230:8181/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/mailman/listinfo/mediawiki-api-announc

P7526
Pandas(Index=7560, property='http://www.wikidata.org/entity/P8422', propType='http://wikiba.se/ontology#ExternalId', propertyLabel='EHESS ID of a French commune', propertyDescription='identifier with digits to describe of French commune by EHESS', datatype='external-id')
P7527
Pandas(Index=7561, property='http://www.wikidata.org/entity/P8425', propType='http://wikiba.se/ontology#ExternalId', propertyLabel='IAPH code', propertyDescription='identifier for cultural heritage records of Instituto Andaluz del Patrimonio Histórico', datatype='external-id')
P7528
Pandas(Index=7562, property='http://www.wikidata.org/entity/P8426', propType='http://wikiba.se/ontology#ExternalId', propertyLabel='Knot Atlas identifier', propertyDescription='link from mathematical knot/link articles to authoritative information source Knot Atlas', datatype='external-id')
P7529
Pandas(Index=7563, property='http://www.wikidata.org/entity/P8427', propType='http://wikiba.se/ontology#ExternalId', propertyLabel='Kn